In [1]:
import time
import pandas as pd
import psycopg2
import datetime
from tqdm import trange
import matplotlib.pyplot as plt 
import matplotlib as mpl 
import seaborn as sns
import math
# %matplotlib inline 
import platform
import numpy as np
from matplotlib import font_manager, rc # plt.rcParams['axes.unicode_minus'] = False 
if platform.system() == 'Darwin': rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': 
    path = "c:/Windows/Fonts/malgun.ttf" 
    font_name = font_manager.FontProperties(fname=path).get_name() 
    rc('font', family=font_name) 
else: 
    print('Unknown system... sorry~~~~')
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
from tqdm import tqdm
from tqdm import trange
from datetime import datetime, timedelta


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


In [3]:
# 시계열
from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing

In [4]:
    # row 생략 없이 출력
pd.set_option('display.max_rows', 1000)
# col 생략 없이 출력
pd.set_option('display.max_columns', 1000)
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
pd.options.display.float_format = '{:.5f}'.format

In [5]:
# 전체 매장 입출판재
total_shop_stock = pd.read_csv("total_shop_stock.csv")
total_shop_stock = total_shop_stock.iloc[ :,1:]

# 매장정보
shop_info_pd = pd.read_csv("shop_info_pd.csv")
shop_info_pd = shop_info_pd.iloc[ :,1:]

# bucket_dwalker 관련 매장입출판재(2019)
bucket_dwalker = pd.read_csv('bucket_dwalker.csv')
bucket_dwalker = bucket_dwalker.iloc[ :,1:]

# bucket_dwalker_v2 관련 매장입출판재
bucket_dwalker_v2 = pd.read_csv('bucket_dwalker_v2.csv')
bucket_dwalker_v2 = bucket_dwalker_v2.iloc[ :,1:]

# 아울렛 상설을 제외한 매장키 
shop_list_pd = pd.read_csv('shop_list.csv')
shop_list_pd = shop_list_pd.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 기준)
shop_weeks_buket_df = pd.read_csv('shop_weeks_buket.csv')
shop_weeks_buket_df = shop_weeks_buket_df.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 v2 기준)
shop_weeks_buket_v2_df = pd.read_csv('shop_weeks_buket_v2.csv')
shop_weeks_buket_v2_df = shop_weeks_buket_v2_df.iloc[ :,1:]

# 전체 매장 비어있는것 채워넣은 입출판재 합쳐 놓은 것
shop_total_sales = pd.read_csv("shop_total_sales.csv")
shop_total_sales = shop_total_sales.iloc[ :,1:]

In [6]:
# DB 연결 부분
uswer = 'dt.intern'
host = '10.80.0.234'
port = '5439'
dbname = 'fnf'
con = psycopg2.connect(dbname='fnf',host='10.80.0.234', port='5439', user = 'dt.intern', password='DTintn0926')
cur = con.cursor()

In [7]:
# 2019년 버킷디워커 물류 수량
start = time.time()

bk_stock_scsweek = '''

SELECT weekkey ,scskey, week_accept_qty ,week_wsl_takeout_qty, wms_stock_qty ,stock_qty,domestic_sale_qty,csm_takeout_qty,csm_sale_qty
FROM ax.vw_f_scsweek vfs 
WHERE scskey  IN ('X19SDXSH09911BE230','X19SDXSH09911BE235','X19SDXSH09911BE240','X19SDXSH09911BE260','X19SDXSH09911BE265','X19SDXSH09911BE270') AND weekkey BETWEEN To_date('2018-12-31', 'YYYY-MM-DD') AND To_date('2019-12-31', 'YYYY-MM-DD')


'''


cur.execute(bk_stock_scsweek)
bk_stock_scsweek = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

bk_stock_scsweek_pd = pd.DataFrame(bk_stock_scsweek)
bk_stock_scsweek_pd.columns = ['weekkey' ,'scskey', 'week_accept_qty' ,'week_wsl_takeout_qty', 'wms_stock_qty' ,'stock_qty','domestic_sale_qty','csm_takeout_qty','csm_sale_qty']

time : 0.16357207298278809


In [8]:
bk_stock_scsweek_pd['csm_qty'] = (bk_stock_scsweek_pd.csm_takeout_qty - bk_stock_scsweek_pd.csm_sale_qty)

In [9]:
bk_stock_scsweek_pd.to_csv('bk_stock_scsweek_pd.csv')

In [10]:
bk_stock_scsweek_pd

,weekkey,scskey,week_accept_qty,week_wsl_takeout_qty,wms_stock_qty,stock_qty,domestic_sale_qty,csm_takeout_qty,csm_sale_qty,csm_qty
0,2018-12-31,X19SDXSH09911BE230,80,0,0,74,6,80,6,74
1,2019-02-25,X19SDXSH09911BE230,640,3,509,675,1214,1380,1214,166
2,2019-03-25,X19SDXSH09911BE230,1238,-9,1146,2328,2359,3541,2359,1182
3,2019-05-20,X19SDXSH09911BE230,0,1,1117,2639,3798,5320,3798,1522
4,2019-06-03,X19SDXSH09911BE230,0,1,996,2501,3935,5440,3935,1505
5,2019-06-17,X19SDXSH09911BE230,0,0,874,2398,4038,5562,4038,1524
6,2019-06-24,X19SDXSH09911BE230,0,0,893,2366,4070,5543,4070,1473
7,2019-07-01,X19SDXSH09911BE230,0,0,789,2313,4123,5647,4123,1524
8,2019-07-29,X19SDXSH09911BE230,0,0,714,2147,4279,5712,4279,1433
9,2019-08-26,X19SDXSH09911BE230,0,0,572,1922,4509,5859,4509,1350


In [11]:
bk_stock_scsweek_pd.groupby('weekkey').sum()[['wms_stock_qty','stock_qty']].reset_index().to_csv('bk_stock_scsweek_pd.csv')

In [12]:
bk_stock_scsweek_pd.groupby('weekkey').sum()[['wms_stock_qty','stock_qty','csm_takeout_qty','csm_sale_qty']].reset_index()

,weekkey,wms_stock_qty,stock_qty,csm_takeout_qty,csm_sale_qty
0,2018-12-31,4,935,956,25
1,2019-01-07,4,869,956,91
2,2019-01-14,849,1537,1501,813
3,2019-01-21,324,807,2095,1612
4,2019-01-28,84,484,2726,2326
5,2019-02-04,199,-97,2880,3176
6,2019-02-11,266,-241,3820,4327
7,2019-02-18,599,1212,6169,5556
8,2019-02-25,2913,3461,7442,6894
9,2019-03-04,663,3042,10923,8544


In [13]:
# 2019년 버킷디워커 물류 수량
start = time.time()

v2_stock_scsweek = '''

SELECT weekkey ,scskey, week_accept_qty ,week_wsl_takeout_qty, wms_stock_qty ,stock_qty,domestic_sale_qty,csm_takeout_qty,csm_sale_qty
FROM ax.vw_f_scsweek vfs 
WHERE scskey  IN ('X20SDXSHA5011OW230','X20SDXSHA5011OW235','X20SDXSHA5011OW240','X20SDXSHA5011OW260','X20SDXSHA5011OW265','X20SDXSHA5011OW270') AND weekkey BETWEEN To_date('2019-11-18', 'YYYY-MM-DD') AND To_date('2020-12-31', 'YYYY-MM-DD')


'''


cur.execute(v2_stock_scsweek)
v2_stock_scsweek = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

v2_stock_scsweek_pd = pd.DataFrame(v2_stock_scsweek)
v2_stock_scsweek_pd.columns = ['weekkey' ,'scskey', 'week_accept_qty' ,'week_wsl_takeout_qty', 'wms_stock_qty' ,'stock_qty','domestic_sale_qty','csm_takeout_qty','csm_sale_qty']

time : 0.1162872314453125


In [14]:
(v2_stock_scsweek_pd.csm_takeout_qty - v2_stock_scsweek_pd.csm_sale_qty ).mean()

895.6553672316384

In [15]:
v2_stock_scsweek_pd['csm_qty'] = (v2_stock_scsweek_pd.csm_takeout_qty - v2_stock_scsweek_pd.csm_sale_qty )

In [16]:
v2_stock_scsweek_pd['csm_qty']

0       343
1       358
2       402
3       414
4       635
5       750
6       626
7       905
8       901
9       929
10      945
11      970
12      994
13      895
14      887
15      875
16      363
17      399
18      520
19      509
20      708
21     1393
22     1344
23     1285
24     1217
25     1264
26     1245
27     1241
28     1239
29     1118
30     1097
31     1104
32      326
33      373
34      422
35      414
36      669
37      903
38      859
39     1050
40     1039
41      986
42     1009
43     1007
44     1030
45      985
46      857
47      837
48      346
49      381
50      460
51      440
52      664
53     1054
54      986
55     1152
56     1097
57     1059
58     1051
59     1031
60     1072
61      977
62      925
63      922
64      348
65      365
66      401
67      382
68      595
69      842
70      804
71     1043
72     1032
73     1033
74     1100
75     1099
76     1093
77     1026
78     1019
79     1020
80      360
81      362
82      411
83  

In [17]:
v2_stock_scsweek_pd.to_csv('v2_stock_scsweek_pd.csv')

In [18]:
v2_stock_scsweek_pd.groupby('weekkey').sum()[['wms_stock_qty','stock_qty']].reset_index().to_csv('v2_stock_scsweek_pd.csv')

In [19]:
v2_stock_scsweek_pd.groupby('weekkey').sum()[['wms_stock_qty','stock_qty']].reset_index()

,weekkey,wms_stock_qty,stock_qty
0,2019-11-18,3224,5199
1,2019-11-25,3350,5436
2,2019-12-02,2611,4849
3,2019-12-09,1941,4277
4,2019-12-16,1087,3516
5,2019-12-23,191,2669
6,2019-12-30,1548,4164
7,2020-01-06,1476,4016
8,2020-01-13,5949,8989
9,2020-01-20,5200,9081


In [20]:
def mae(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    abs_val = abs(difference)
    
    score = abs_val.mean()
    
    return score

# mae_scorer = make_scorer(mae)
# mae_scorer

In [21]:
def make_ewm_arr(data_frame, span_num):
    arr_ewm = np.array([])
#     for i in data_frame.store_id.unique():
    df_set = data_frame
        # 여기에서 지정하는 span값은 위 수식에서 N에 해당한다.
    arr_ewm = np.concatenate((arr_ewm, df_set.week_sale_qty.ewm(span=span_num).mean().values) )
    
    return arr_ewm

# 재고가 0 인 지점

## 2020년 버킷디워커 v2

In [22]:
shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='백화점']

In [23]:
shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey'] != 'X10047']

In [24]:
v2_shopkey_unique =  shop_weeks_buket_v2_df_dep.shopkey.unique()
len(v2_shopkey_unique)

66

In [25]:
v2_scskey_unique =  shop_weeks_buket_v2_df_dep.scskey.unique()
len(v2_scskey_unique)

6

In [26]:
v2_shop_qty = pd.DataFrame()

for i in v2_shopkey_unique:
    uniqe_shop = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey']==i]
    for j in v2_scskey_unique:
        uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
        ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
        uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
        ewm_value = np.append([0],ewm_value,axis=0)
        uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
        for i in range(0,len(ewm_value)):
#             ewm_value[i] = round(ewm_value[i] , 0)+1
            ewm_value[i] = math.ceil(ewm_value[i])+1

        uniqe_shop_scskey['qty'] = ewm_value[:-1]
        v2_shop_qty = pd.concat([v2_shop_qty, uniqe_shop_scskey], axis=0)

<ipython-input-26-7d2f0dd62d4a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-26-7d2f0dd62d4a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]


In [27]:
v2_shop_qty.groupby('weekkey').sum()

,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,ratio,amount_2ewm,amount_2ewm_week,qty
weekkey,,,,,,,,,
2019-11-18,732,732,98,98,634,2.62746,98.00000,0.00000,396.00000
2019-11-25,249,981,191,289,692,8.12039,167.75000,98.00000,494.00000
2019-12-02,345,1326,257,546,780,15.77649,229.53846,167.75000,634.00000
2019-12-09,298,1624,260,806,818,22.43627,250.10000,229.53846,753.00000
2019-12-16,454,2078,379,1185,893,34.00044,336.38843,250.10000,813.00000
2019-12-23,329,2407,363,1548,859,44.05726,354.15385,336.38843,919.00000
2019-12-30,377,2784,290,1838,946,52.27536,311.36505,354.15385,950.00000
2020-01-06,320,3104,329,2167,937,61.76619,323.12348,311.36505,917.00000
2020-01-13,494,3598,285,2452,1146,69.38468,297.70653,323.12348,930.00000


In [28]:
v2_shop_qty

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,amount_2ewm,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm,amount_2ewm_week,qty
0,2019-11-18,1,1,0,0,1,X10056,0.00000,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.00000,XH0247,일반매장,백화점특정,백화점,경남,0.00000,1.00000
1,2019-11-25,0,1,0,0,1,X10056,0.00000,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.00000,XH0247,일반매장,백화점특정,백화점,경남,0.00000,1.00000
2,2019-12-02,2,3,1,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.69231,XH0247,일반매장,백화점특정,백화점,경남,0.00000,1.00000
3,2019-12-09,0,3,0,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.22500,XH0247,일반매장,백화점특정,백화점,경남,0.69231,2.00000
4,2019-12-16,0,3,0,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.07438,XH0247,일반매장,백화점특정,백화점,경남,0.22500,2.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50499,2020-11-30,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,0.11709,XH0237,일반매장,백화점특정,백화점,경남,0.35127,2.00000
50500,2020-12-07,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,0.03903,XH0237,일반매장,백화점특정,백화점,경남,0.11709,2.00000
50501,2020-12-14,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,0.01301,XH0237,일반매장,백화점특정,백화점,경남,0.03903,2.00000
50502,2020-12-21,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,0.00434,XH0237,일반매장,백화점특정,백화점,경남,0.01301,2.00000


In [29]:
v2_zero = v2_shop_qty[(v2_shop_qty['stock_qty']==0) & (v2_shop_qty['takeout_qty']>0)]

In [30]:
len(v2_zero)

240

In [31]:
len(v2_zero[v2_zero['weekkey'] < '2020-11-01'])

205

In [32]:
# 재고에서 0 이 아닌 부분
len(v2_zero[v2_zero.qty - v2_zero.week_sale_qty>0])

78

In [33]:
# 재고에서 0 인 부분 
len(v2_zero[v2_zero.qty - v2_zero.week_sale_qty<=0])

162

In [34]:
shop_weeks_buket_v2_df_dep

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
0,2019-11-18,1,1,0,0,1,X10056,0.00000,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
1,2019-11-25,0,1,0,0,1,X10056,0.00000,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
2,2019-12-02,2,3,1,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
3,2019-12-09,0,3,0,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
4,2019-12-16,0,3,0,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50499,2020-11-30,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50500,2020-12-07,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50501,2020-12-14,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50502,2020-12-21,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남


In [35]:
frame = shop_weeks_buket_v2_df_dep[:59][['weekkey','stock_qty']]

In [36]:
v2_zero_modi = v2_zero[v2_zero.qty - v2_zero.week_sale_qty<=0].groupby(['weekkey']).count().qty.reset_index()

In [37]:
v2_zero[v2_zero.qty - v2_zero.week_sale_qty<=0].groupby(['weekkey']).count().qty.reset_index().to_csv('v2_zero_modi.csv')

In [38]:
v2_zero_modi_frame = shop_weeks_buket_v2_df_dep[:59][['weekkey','stock_qty']]
v2_zero_modi_frame.stock_qty = 0
for i in range(0,len(v2_zero_modi_frame)):
    for j in range(0,len(v2_zero_modi)):
        if v2_zero_modi_frame.iloc[i].weekkey == v2_zero_modi.iloc[j].weekkey:
            v2_zero_modi_frame.iloc[i] = [v2_zero_modi.iloc[j].weekkey , v2_zero_modi.iloc[j].qty]

In [39]:
v2_zero_modi_frame.to_csv('v2_zero_modi_frame.csv')

In [40]:
v2_zero_ori = v2_zero.groupby(['weekkey']).count().qty.reset_index()

In [41]:
v2_zero_ori_frame = shop_weeks_buket_v2_df_dep[:59][['weekkey','stock_qty']]
v2_zero_ori_frame.stock_qty = 0

for i in range(0,len(v2_zero_ori_frame)):
    for j in range(0,len(v2_zero_ori)):
        if v2_zero_ori_frame.iloc[i].weekkey == v2_zero_ori.iloc[j].weekkey:
            v2_zero_ori_frame.iloc[i] = [v2_zero_ori.iloc[j].weekkey , v2_zero_ori.iloc[j].qty]

In [42]:
v2_zero_ori_frame.to_csv('v2_zero_ori_frame.csv')

In [43]:
v2_merge_outer = pd.merge(v2_zero_ori_frame,v2_zero_modi_frame, how='outer',on='weekkey')
v2_merge_outer.to_csv('v2_merge_outer.csv')

In [44]:
v2_merge_outer.to_csv('v2_merge_outer.csv')

In [45]:
v2_zero[v2_zero.qty - v2_zero.week_sale_qty<=0].groupby(['weekkey','scskey']).count().qty.reset_index()

,weekkey,scskey,qty
0,2019-11-18,X20SDXSHA5011OW230,3
1,2019-11-18,X20SDXSHA5011OW235,5
2,2019-11-18,X20SDXSHA5011OW240,2
3,2019-11-18,X20SDXSHA5011OW260,4
4,2019-11-18,X20SDXSHA5011OW265,4
5,2019-11-18,X20SDXSHA5011OW270,1
6,2019-11-25,X20SDXSHA5011OW230,1
7,2019-11-25,X20SDXSHA5011OW235,7
8,2019-11-25,X20SDXSHA5011OW240,3
9,2019-11-25,X20SDXSHA5011OW260,4


In [46]:
v2_zero[v2_zero.qty - v2_zero.week_sale_qty<=0].groupby(['weekkey']).count().qty.reset_index()

,weekkey,qty
0,2019-11-18,19
1,2019-11-25,22
2,2019-12-02,18
3,2019-12-09,14
4,2019-12-16,12
5,2019-12-23,10
6,2019-12-30,5
7,2020-01-06,9
8,2020-01-13,4
9,2020-01-20,2


In [47]:
v2_zero.groupby(['weekkey']).count().qty.reset_index()

,weekkey,qty
0,2019-11-18,19
1,2019-11-25,23
2,2019-12-02,19
3,2019-12-09,19
4,2019-12-16,13
5,2019-12-23,15
6,2019-12-30,11
7,2020-01-06,11
8,2020-01-13,6
9,2020-01-20,3


In [48]:
v2_zero.groupby(['weekkey']).count().qty.reset_index().to_csv('v2_zero_ori.csv')

In [49]:
shop_weeks_buket_v2_df_dep.groupby(['weekkey']).sum().stock_qty.to_csv('v2_shop_qty.csv')

In [50]:
shop_weeks_buket_v2_df_dep.groupby(['weekkey']).sum().stock_qty.mean()

1750.457627118644

In [51]:
v2_stock_scsweek_pd.groupby(['weekkey']).sum().wms_stock_qty

weekkey
2019-11-18     3224
2019-11-25     3350
2019-12-02     2611
2019-12-09     1941
2019-12-16     1087
2019-12-23      191
2019-12-30     1548
2020-01-06     1476
2020-01-13     5949
2020-01-20     5200
2020-01-27     4438
2020-02-03     3321
2020-02-10     1793
2020-02-17     1822
2020-02-24     1554
2020-03-02     1634
2020-03-09     3839
2020-03-16     7095
2020-03-23     7408
2020-03-30     7073
2020-04-06     6371
2020-04-13     8698
2020-04-20    10118
2020-04-27     9511
2020-05-04    11392
2020-05-11    12355
2020-05-18    11815
2020-05-25    11428
2020-06-01    10796
2020-06-08    10410
2020-06-15    10088
2020-06-22    10079
2020-06-29    10187
2020-07-06    11311
2020-07-13    14779
2020-07-20    14512
2020-07-27    12992
2020-08-03    12713
2020-08-10    12467
2020-08-17    12191
2020-08-24    11962
2020-08-31    11249
2020-09-07    10951
2020-09-14    10678
2020-09-21    10217
2020-09-28     9928
2020-10-05     9162
2020-10-12     8480
2020-10-19     7986
2020-10-26  

In [52]:
v2_stock_scsweek_pd.groupby(['weekkey']).sum().wms_stock_qty.mean()

7225.745762711865

In [53]:
v2_zero.groupby(['weekkey','scskey']).count().qty.reset_index()

,weekkey,scskey,qty
0,2019-11-18,X20SDXSHA5011OW230,3
1,2019-11-18,X20SDXSHA5011OW235,5
2,2019-11-18,X20SDXSHA5011OW240,2
3,2019-11-18,X20SDXSHA5011OW260,4
4,2019-11-18,X20SDXSHA5011OW265,4
5,2019-11-18,X20SDXSHA5011OW270,1
6,2019-11-25,X20SDXSHA5011OW230,1
7,2019-11-25,X20SDXSHA5011OW235,7
8,2019-11-25,X20SDXSHA5011OW240,3
9,2019-11-25,X20SDXSHA5011OW260,5


In [54]:
shop_weeks_buket_v2_df_dep.groupby(['weekkey','scskey']).sum().stock_qty.reset_index()

,weekkey,scskey,stock_qty
0,2019-11-18,X20SDXSHA5011OW230,108
1,2019-11-18,X20SDXSHA5011OW235,102
2,2019-11-18,X20SDXSHA5011OW240,101
3,2019-11-18,X20SDXSHA5011OW260,105
4,2019-11-18,X20SDXSHA5011OW265,108
5,2019-11-18,X20SDXSHA5011OW270,110
6,2019-11-25,X20SDXSHA5011OW230,114
7,2019-11-25,X20SDXSHA5011OW235,111
8,2019-11-25,X20SDXSHA5011OW240,123
9,2019-11-25,X20SDXSHA5011OW260,106


In [55]:
v2_stock_scsweek_pd.groupby(['weekkey','scskey']).sum().wms_stock_qty.reset_index()

,weekkey,scskey,wms_stock_qty
0,2019-11-18,X20SDXSHA5011OW230,404
1,2019-11-18,X20SDXSHA5011OW235,551
2,2019-11-18,X20SDXSHA5011OW240,911
3,2019-11-18,X20SDXSHA5011OW260,424
4,2019-11-18,X20SDXSHA5011OW265,423
5,2019-11-18,X20SDXSHA5011OW270,511
6,2019-11-25,X20SDXSHA5011OW230,394
7,2019-11-25,X20SDXSHA5011OW235,530
8,2019-11-25,X20SDXSHA5011OW240,885
9,2019-11-25,X20SDXSHA5011OW260,446


## 2019년 버킷디워커

In [56]:
shop_weeks_buket_df_dep = shop_weeks_buket_df[shop_weeks_buket_df['type_zone_nm']=='백화점']

In [57]:
shop_weeks_buket_df_dep

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
0,2018-12-31,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
1,2019-01-07,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
2,2019-01-14,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
3,2019-01-21,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
4,2019-01-28,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45363,2019-12-02,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
45364,2019-12-09,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
45365,2019-12-16,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
45366,2019-12-23,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남


In [58]:
bk_shopkey_unique =  shop_weeks_buket_df_dep.shopkey.unique()
len(bk_shopkey_unique)

67

In [59]:
bk_scskey_unique =  shop_weeks_buket_df_dep.scskey.unique()
len(bk_scskey_unique)

6

In [60]:
shop_weeks_buket_df_dep = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey'] != 'X10047']

In [61]:
bk_shop_qty = pd.DataFrame()

for i in bk_shopkey_unique:
    uniqe_shop = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey']==i]
    for j in bk_scskey_unique:
        uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
        ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
        uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
        ewm_value = np.append([0],ewm_value,axis=0)
        uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
        for i in range(0,len(ewm_value)):
#             ewm_value[i] = round(ewm_value[i] , 0)+1
            ewm_value[i] = math.ceil(ewm_value[i])+1

        uniqe_shop_scskey['qty'] = ewm_value[:-1]
        bk_shop_qty = pd.concat([bk_shop_qty, uniqe_shop_scskey], axis=0)

<ipython-input-61-d32067fabf12>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-61-d32067fabf12>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]


In [62]:
bk_shop_qty.groupby('weekkey').sum().stock_qty.mean()

1801.9622641509434

In [63]:
bk_shop_qty.groupby('weekkey').sum().stock_qty.to_csv('bk_shop_qty.csv')

In [64]:
bk_zero = bk_shop_qty[(bk_shop_qty['stock_qty']==0) & (bk_shop_qty['takeout_qty']>0)]

In [65]:
bk_zero

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,amount_2ewm,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm,amount_2ewm_week,qty
6,2019-02-11,1,1,0,1,0,X10056,0.08333,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,0.22232,XH0247,일반매장,백화점특정,백화점,경남,0.66758,2.00000
15162,2019-01-28,-1,2,0,2,0,X10056,0.10526,X19SDXSH09911BE240,갤러리아진주,디스커버리 갤러리아 진주점,0.29752,XH0247,일반매장,백화점특정,백화점,경남,0.90000,2.00000
15163,2019-02-04,0,2,0,2,0,X10056,0.10526,X19SDXSH09911BE240,갤러리아진주,디스커버리 갤러리아 진주점,0.09890,XH0247,일반매장,백화점특정,백화점,경남,0.29752,2.00000
15164,2019-02-11,0,2,0,2,0,X10056,0.10526,X19SDXSH09911BE240,갤러리아진주,디스커버리 갤러리아 진주점,0.03294,XH0247,일반매장,백화점특정,백화점,경남,0.09890,2.00000
15166,2019-02-25,0,3,1,3,0,X10056,0.15789,X19SDXSH09911BE240,갤러리아진주,디스커버리 갤러리아 진주점,0.67036,XH0247,일반매장,백화점특정,백화점,경남,0.01098,2.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45363,2019-12-02,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,0.00007,XH0237,일반매장,백화점특정,백화점,경남,0.00020,2.00000
45364,2019-12-09,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,0.00002,XH0237,일반매장,백화점특정,백화점,경남,0.00007,2.00000
45365,2019-12-16,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,0.00001,XH0237,일반매장,백화점특정,백화점,경남,0.00002,2.00000
45366,2019-12-23,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,0.00000,XH0237,일반매장,백화점특정,백화점,경남,0.00001,2.00000


In [66]:
len(bk_zero)

2225

In [199]:
test3 = bk_shop_qty[(bk_shop_qty['shop_nm']=='롯데본점')&(bk_shop_qty['weekkey']>='2019-03-10')][['weekkey','scskey','week_sale_qty','week_takeout_qty','stock_qty','amount_2ewm_week','qty','shop_nm']]

In [200]:
test3['qty'] = pd.to_numeric(test3['qty'])
test3= test3.convert_dtypes()

In [202]:
test3.columns = ['주차','scskey','주 판매량','_　　　　_주 입고량','_　　　　_재고량','_　　　　_지수 이동 평균값','_　　　　_차주 최소 재고량','매장이름']
test3

,주차,scskey,주 판매량,_ _주 입고량,_ _재고량,_ _지수 이동 평균값,_ _차주 최소 재고량,매장이름
2342,2019-03-11,X19SDXSH09911BE230,1,3,4,0.07408,2,롯데본점
2343,2019-03-18,X19SDXSH09911BE230,0,0,4,0.69136,2,롯데본점
2344,2019-03-25,X19SDXSH09911BE230,2,3,5,0.23045,2,롯데본점
2345,2019-04-01,X19SDXSH09911BE230,3,4,6,1.41015,3,롯데본점
2346,2019-04-08,X19SDXSH09911BE230,1,0,5,2.47005,4,롯데본점
2347,2019-04-15,X19SDXSH09911BE230,3,6,8,1.49002,3,롯데본점
2348,2019-04-22,X19SDXSH09911BE230,3,1,6,2.49667,4,롯데본점
2349,2019-04-29,X19SDXSH09911BE230,2,4,8,2.83222,4,롯데본점
2350,2019-05-06,X19SDXSH09911BE230,-1,0,9,2.27741,4,롯데본점
2351,2019-05-13,X19SDXSH09911BE230,1,0,8,0.09247,2,롯데본점


In [67]:
# 재고에서 0 이 지워진 부분 
len(bk_zero[bk_zero.qty - bk_zero.week_sale_qty>0])

1863

In [68]:
# 아직 재고가 0 이하인 부분
len(bk_zero[bk_zero.qty - bk_zero.week_sale_qty<=0])

362

In [69]:
# zero 인 지점
bk_zero.groupby(['weekkey','scskey']).count().qty.reset_index()

,weekkey,scskey,qty
0,2018-12-31,X19SDXSH09911BE235,1
1,2018-12-31,X19SDXSH09911BE240,1
2,2018-12-31,X19SDXSH09911BE260,2
3,2018-12-31,X19SDXSH09911BE270,1
4,2019-01-07,X19SDXSH09911BE230,1
5,2019-01-07,X19SDXSH09911BE235,3
6,2019-01-07,X19SDXSH09911BE240,2
7,2019-01-07,X19SDXSH09911BE260,6
8,2019-01-07,X19SDXSH09911BE265,2
9,2019-01-07,X19SDXSH09911BE270,4


In [70]:
# stock_qty 재고 개수
shop_weeks_buket_df_dep.groupby(['weekkey','scskey']).sum().stock_qty.reset_index()

,weekkey,scskey,stock_qty
0,2018-12-31,X19SDXSH09911BE230,15
1,2018-12-31,X19SDXSH09911BE235,63
2,2018-12-31,X19SDXSH09911BE240,79
3,2018-12-31,X19SDXSH09911BE260,62
4,2018-12-31,X19SDXSH09911BE265,17
5,2018-12-31,X19SDXSH09911BE270,63
6,2019-01-07,X19SDXSH09911BE230,12
7,2019-01-07,X19SDXSH09911BE235,59
8,2019-01-07,X19SDXSH09911BE240,74
9,2019-01-07,X19SDXSH09911BE260,58


In [71]:
bk_stock_scsweek_pd.groupby(['weekkey','scskey']).sum().wms_stock_qty.reset_index()

,weekkey,scskey,wms_stock_qty
0,2018-12-31,X19SDXSH09911BE230,0
1,2018-12-31,X19SDXSH09911BE235,1
2,2018-12-31,X19SDXSH09911BE240,1
3,2018-12-31,X19SDXSH09911BE260,1
4,2018-12-31,X19SDXSH09911BE265,0
5,2018-12-31,X19SDXSH09911BE270,1
6,2019-01-07,X19SDXSH09911BE230,0
7,2019-01-07,X19SDXSH09911BE235,1
8,2019-01-07,X19SDXSH09911BE240,1
9,2019-01-07,X19SDXSH09911BE260,1


In [72]:
bk_zero_modi = bk_zero[bk_zero.qty - bk_zero.week_sale_qty<=0].groupby(['weekkey']).count().qty.reset_index()

In [73]:
bk_zero_ori = bk_zero.groupby(['weekkey']).count().qty.reset_index()

In [74]:
bk_zero_modi_frame = shop_weeks_buket_df_dep[:53][['weekkey','stock_qty']]
bk_zero_modi_frame.stock_qty = 0

for i in range(0,len(bk_zero_modi_frame)):
    for j in range(0,len(bk_zero_modi)):
        if bk_zero_modi_frame.iloc[i].weekkey == bk_zero_modi.iloc[j].weekkey:
            bk_zero_modi_frame.iloc[i] = [bk_zero_modi.iloc[j].weekkey , bk_zero_modi.iloc[j].qty]
bk_zero_modi_frame.to_csv('bk_zero_modi_frame.csv')

In [75]:
bk_zero_ori_frame = shop_weeks_buket_df_dep[:53][['weekkey','stock_qty']]
bk_zero_ori_frame.stock_qty = 0

for i in range(0,len(bk_zero_ori_frame)):
    for j in range(0,len(bk_zero_ori)):
        if bk_zero_ori_frame.iloc[i].weekkey == bk_zero_ori.iloc[j].weekkey:
            bk_zero_ori_frame.iloc[i] = [bk_zero_ori.iloc[j].weekkey , bk_zero_ori.iloc[j].qty]
bk_zero_ori_frame.to_csv('bk_zero_ori_frame.csv')

In [76]:
bk_merge_outer = pd.merge(bk_zero_ori_frame,bk_zero_modi_frame, how='outer',on='weekkey')
bk_merge_outer.to_csv('bk_merge_outer.csv')

In [77]:
bk_merge_outer

,weekkey,stock_qty_x,stock_qty_y
0,2018-12-31,5,5
1,2019-01-07,18,16
2,2019-01-14,76,74
3,2019-01-21,109,32
4,2019-01-28,103,16
5,2019-02-04,98,26
6,2019-02-11,124,7
7,2019-02-18,103,20
8,2019-02-25,150,42
9,2019-03-04,64,18


In [135]:
v2_shop_qty[v2_shop_qty.shop_nm == '현대판교']

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,amount_2ewm,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm,amount_2ewm_week,qty
1711,2019-11-18,1,1,0,0,1,X10068,0.00000,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.00000,XH0257,일반매장,백화점특정,백화점,경기,0.00000,1.00000
1712,2019-11-25,0,1,0,0,1,X10068,0.00000,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.00000,XH0257,일반매장,백화점특정,백화점,경기,0.00000,1.00000
1713,2019-12-02,0,1,0,0,1,X10068,0.00000,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.00000,XH0257,일반매장,백화점특정,백화점,경기,0.00000,1.00000
1714,2019-12-09,0,1,0,0,1,X10068,0.00000,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.00000,XH0257,일반매장,백화점특정,백화점,경기,0.00000,1.00000
1715,2019-12-16,15,16,15,15,1,X10068,0.57692,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,10.04132,XH0257,일반매장,백화점특정,백화점,경기,0.00000,1.00000
1716,2019-12-23,0,16,1,16,0,X10068,0.61538,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,4.00549,XH0257,일반매장,백화점특정,백화점,경기,10.04132,12.00000
1717,2019-12-30,4,20,0,16,4,X10068,0.61538,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,1.33394,XH0257,일반매장,백화점특정,백화점,경기,4.00549,6.00000
1718,2020-01-06,0,20,0,16,4,X10068,0.61538,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.44451,XH0257,일반매장,백화점특정,백화점,경기,1.33394,3.00000
1719,2020-01-13,0,20,1,17,3,X10068,0.65385,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.81486,XH0257,일반매장,백화점특정,백화점,경기,0.44451,2.00000
1720,2020-01-20,2,22,0,17,5,X10068,0.65385,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.27161,XH0257,일반매장,백화점특정,백화점,경기,0.81486,2.00000


In [130]:
v2_shop_qty[(v2_shop_qty.shop_nm == '신세계마산')&(v2_shop_qty.scskey == 'X20SDXSHA5011OW270')]

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,amount_2ewm,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm,amount_2ewm_week,qty
45194,2019-11-18,2,2,1,1,1,X10073,0.05556,X20SDXSHA5011OW270,신세계마산,디스커버리 신세계 마산점,1.00000,XH264,일반매장,백화점특정,백화점,경남,0.00000,1.00000
45195,2019-11-25,1,3,-1,0,3,X10073,0.00000,X20SDXSHA5011OW270,신세계마산,디스커버리 신세계 마산점,-0.50000,XH264,일반매장,백화점특정,백화점,경남,1.00000,2.00000
45196,2019-12-02,0,3,0,0,3,X10073,0.00000,X20SDXSHA5011OW270,신세계마산,디스커버리 신세계 마산점,-0.15385,XH264,일반매장,백화점특정,백화점,경남,-0.50000,1.00000
45197,2019-12-09,0,3,0,0,3,X10073,0.00000,X20SDXSHA5011OW270,신세계마산,디스커버리 신세계 마산점,-0.05000,XH264,일반매장,백화점특정,백화점,경남,-0.15385,1.00000
45198,2019-12-16,0,3,1,1,2,X10073,0.05556,X20SDXSHA5011OW270,신세계마산,디스커버리 신세계 마산점,0.65289,XH264,일반매장,백화점특정,백화점,경남,-0.05000,1.00000
45199,2019-12-23,1,4,1,2,2,X10073,0.11111,X20SDXSHA5011OW270,신세계마산,디스커버리 신세계 마산점,0.88462,XH264,일반매장,백화점특정,백화점,경남,0.65289,2.00000
45200,2019-12-30,0,4,0,2,2,X10073,0.11111,X20SDXSHA5011OW270,신세계마산,디스커버리 신세계 마산점,0.29460,XH264,일반매장,백화점특정,백화점,경남,0.88462,2.00000
45201,2020-01-06,0,4,0,2,2,X10073,0.11111,X20SDXSHA5011OW270,신세계마산,디스커버리 신세계 마산점,0.09817,XH264,일반매장,백화점특정,백화점,경남,0.29460,2.00000
45202,2020-01-13,0,4,1,3,1,X10073,0.16667,X20SDXSHA5011OW270,신세계마산,디스커버리 신세계 마산점,0.69942,XH264,일반매장,백화점특정,백화점,경남,0.09817,2.00000
45203,2020-01-20,3,7,0,3,4,X10073,0.16667,X20SDXSHA5011OW270,신세계마산,디스커버리 신세계 마산점,0.23313,XH264,일반매장,백화점특정,백화점,경남,0.69942,2.00000


In [151]:
test2 = v2_shop_qty[(v2_shop_qty.shop_nm == '롯데평촌')&(v2_shop_qty.scskey == 'X20SDXSHA5011OW270')&(v2_shop_qty.weekkey >'2019-12-08')][['weekkey','week_sale_qty','week_takeout_qty','scskey','stock_qty','shop_nm']]

In [162]:
test2.columns = [['주차','주 판매량','_　　　　_주 입고량','scskey','재고량','매장이름']]
test2

,주차,주 판매량,_ _주 입고량,scskey,재고량,매장이름
42896,2019-12-09,2,4,X20SDXSHA5011OW270,2,롯데평촌
42897,2019-12-16,0,1,X20SDXSHA5011OW270,3,롯데평촌
42898,2019-12-23,1,1,X20SDXSHA5011OW270,3,롯데평촌
42899,2019-12-30,1,0,X20SDXSHA5011OW270,2,롯데평촌
42900,2020-01-06,1,2,X20SDXSHA5011OW270,3,롯데평촌
42901,2020-01-13,0,0,X20SDXSHA5011OW270,3,롯데평촌
42902,2020-01-20,0,0,X20SDXSHA5011OW270,3,롯데평촌
42903,2020-01-27,3,1,X20SDXSHA5011OW270,1,롯데평촌
42904,2020-02-03,2,3,X20SDXSHA5011OW270,2,롯데평촌
42905,2020-02-10,4,3,X20SDXSHA5011OW270,1,롯데평촌


In [141]:
test2

,weekkey,week_takeout_qty,week_sale_qty,stock_qty,scskey,shop_nm
42898,2019-12-23,1,1,3,X20SDXSHA5011OW270,롯데평촌
42899,2019-12-30,0,1,2,X20SDXSHA5011OW270,롯데평촌
42900,2020-01-06,2,1,3,X20SDXSHA5011OW270,롯데평촌
42901,2020-01-13,0,0,3,X20SDXSHA5011OW270,롯데평촌
42902,2020-01-20,0,0,3,X20SDXSHA5011OW270,롯데평촌
42903,2020-01-27,1,3,1,X20SDXSHA5011OW270,롯데평촌
42904,2020-02-03,3,2,2,X20SDXSHA5011OW270,롯데평촌
42905,2020-02-10,3,4,1,X20SDXSHA5011OW270,롯데평촌
42906,2020-02-17,1,1,1,X20SDXSHA5011OW270,롯데평촌
42907,2020-02-24,1,1,1,X20SDXSHA5011OW270,롯데평촌


In [116]:
v2_shop_qty.shop_nm.unique()

array(['갤러리아진주', '갤러리아센터시티', '롯데평촌', '신세계강남', '신세계대구', '신세계스타일마켓',
       '아이파크용산', '현대대구', '현대무역', '현대울산', '롯데광복', '롯데대전', '롯데상인', '현대판교',
       '롯데안산', '롯데일산', '롯데강남', '롯데건대', '롯데본점', '롯데노원', '롯데포항', '신세계경기',
       '신세계마산', '신세계의정부', '엔터6왕십리', 'AK수원', '현대미아', '현대울산동구', '롯데광주',
       '롯데동래', '현대천호', '롯데영등포', '롯데중동', 'AK원주', '갤러리아대전', '롯데전주', '롯데미아',
       '세이대전', '신세계김해', '신세계센텀', '신세계하남', 'AK분당', '현대목동', '현대신촌', '롯데관악',
       '롯데김포', '롯데분당', '현대킨텍스', '롯데수원', '롯데울산', '롯데청량리', 'LF스퀘어광양',
       '롯데마산', '롯데잠실', '롯데대구', '신세계광주', '신세계본점', '신세계아라리오', '현대디큐브시티',
       '현대부산', '현대중동', '롯데구리', '롯데부산', '현대충청', '롯데센텀', '롯데창원'],
      dtype=object)

In [79]:
sum(v2_shop_qty[v2_shop_qty['shop_nm']=='갤러리아진주'].stock_qty - v2_shop_qty[v2_shop_qty['shop_nm']=='갤러리아진주'].qty)

257.0

In [80]:
test1 = v2_shop_qty.copy()

In [81]:
test1['stock_qty'] = test1.stock_qty - test1.qty

In [82]:
test1.groupby('weekkey').sum().stock_qty

weekkey
2019-11-18    238.00000
2019-11-25    198.00000
2019-12-02    146.00000
2019-12-09     65.00000
2019-12-16     80.00000
2019-12-23    -60.00000
2019-12-30     -4.00000
2020-01-06     20.00000
2020-01-13    216.00000
2020-01-20    475.00000
2020-01-27    548.00000
2020-02-03    515.00000
2020-02-10    393.00000
2020-02-17    420.00000
2020-02-24    531.00000
2020-03-02    565.00000
2020-03-09    490.00000
2020-03-16    495.00000
2020-03-23    603.00000
2020-03-30    684.00000
2020-04-06    713.00000
2020-04-13    830.00000
2020-04-20    886.00000
2020-04-27    859.00000
2020-05-04   1031.00000
2020-05-11   1088.00000
2020-05-18   1134.00000
2020-05-25   1069.00000
2020-06-01   1198.00000
2020-06-08   1228.00000
2020-06-15   1260.00000
2020-06-22   1280.00000
2020-06-29   1295.00000
2020-07-06   1307.00000
2020-07-13   1313.00000
2020-07-20   1316.00000
2020-07-27   1338.00000
2020-08-03   1352.00000
2020-08-10   1337.00000
2020-08-17   1363.00000
2020-08-24   1381.00000
2020-08-

In [83]:
mae(test1.week_takeout_qty,test1.qty)

1.6996233521657251

In [84]:
mae(test1.week_takeout_qty,test1.qty)

1.6996233521657251

In [85]:
mae(test1.week_takeout_qty[1:],test1.stock_qty[:-1])

2.498908530582545

In [86]:
shopkey_unique = test1.shopkey.unique()
scskey_unique = test1.scskey.unique()
mae_list=[]
for i in shopkey_unique:
    for j in scskey_unique:
#         print(test1[(test1['shopkey']==i) & (test1['scskey']==j)].week_takeout_qty[1:])
        mae_list.append(mae( test1[(test1['shopkey']==i) & (test1['scskey']==j)].week_takeout_qty[1:] ,test1[(test1['shopkey']==i) & (test1['scskey']==j)].stock_qty[:-1] ))
#         break
#     break
mae_list
        
    

[1.0689655172413792,
 1.1206896551724137,
 1.293103448275862,
 1.1896551724137931,
 1.6724137931034482,
 1.0,
 2.2586206896551726,
 3.5172413793103448,
 3.1724137931034484,
 0.9137931034482759,
 2.5344827586206895,
 1.8620689655172413,
 4.379310344827586,
 3.4482758620689653,
 2.603448275862069,
 3.206896551724138,
 3.1206896551724137,
 4.086206896551724,
 1.4482758620689655,
 2.4827586206896552,
 2.5517241379310347,
 1.0517241379310345,
 2.7413793103448274,
 1.7241379310344827,
 2.396551724137931,
 3.810344827586207,
 3.810344827586207,
 2.1379310344827585,
 2.1206896551724137,
 2.3620689655172415,
 1.7413793103448276,
 1.603448275862069,
 1.2241379310344827,
 0.896551724137931,
 1.3275862068965518,
 1.1551724137931034,
 1.5172413793103448,
 3.396551724137931,
 1.9482758620689655,
 1.103448275862069,
 2.086206896551724,
 1.5517241379310345,
 1.9655172413793103,
 2.0344827586206895,
 2.793103448275862,
 2.6551724137931036,
 2.5517241379310347,
 1.6896551724137931,
 1.6206896551724137,


In [87]:
np.mean(mae_list)

2.5141501219087425

In [96]:
bucket_dwalker[(bucket_dwalker['stock_qty']==0)&(bucket_dwalker['color']=='BE')].scskey.unique()

array(['X19SDXSH09911BE245', 'X19SDXSH09911BE230', 'X19SDXSH09911BE250',
       'X19SDXSH09911BE265', 'X19SDXSH09911BE290', 'X19SDXSH09911BE240',
       'X19SDXSH09911BE260', 'X19SDXSH09911BE280', 'X19SDXSH09911BE255',
       'X19SDXSH09911BE270', 'X19SDXSH09911BE235', 'X19SDXSH09911BE225',
       'X19SDXSH09911BE275'], dtype=object)

In [101]:
bucket_dwalker_v2[(bucket_dwalker_v2['stock_qty']==0)&(bucket_dwalker_v2['color']=='OW')].count()

shopkey             2049
weekkey             2049
partkey             2049
scskey              2049
brand               2049
season              2049
partcode            2049
color               2049
siz                 2049
week_takeout_qty    2049
takeout_qty         2049
week_sale_qty       2049
sale_qty            2049
week_sale_amt       2049
stock_qty           2049
sale_amt            2049
gender              2049
dtype: int64

In [104]:
bucket_dwalker_v2[(bucket_dwalker_v2['stock_qty']==0)&(bucket_dwalker_v2['color']=='OW')].scskey.unique()

array(['X20SDXSHA5011OW230', 'X20SDXSHA5011OW275', 'X20SDXSHA5011OW245',
       'X20SDXSHA5011OW280', 'X20SDXSHA5011OW260', 'X20SDXSHA5011OW270',
       'X20SDXSHA5011OW290', 'X20SDXSHA5011OW265', 'X20SDXSHA5011OW255',
       'X20SDXSHA5011OW285', 'X20SDXSHA5011OW240', 'X20SDXSHA5011OW225',
       'X20SDXSHA5011OW250', 'X20SDXSHA5011OW235'], dtype=object)

In [108]:
size_list = ['X20SDXSHA5011OW230','X20SDXSHA5011OW235','X20SDXSHA5011OW240','X20SDXSHA5011OW260','X20SDXSHA5011OW265','X20SDXSHA5011OW270']
count_list = []
for i in size_list:
    count_list.append(bucket_dwalker_v2[(bucket_dwalker_v2['stock_qty']==0)&(bucket_dwalker_v2['color']=='OW')&(bucket_dwalker_v2['scskey']==i)].count())

sum(count_list)

shopkey             432
weekkey             432
partkey             432
scskey              432
brand               432
season              432
partcode            432
color               432
siz                 432
week_takeout_qty    432
takeout_qty         432
week_sale_qty       432
sale_qty            432
week_sale_amt       432
stock_qty           432
sale_amt            432
gender              432
dtype: int64

In [110]:
bucket_dwalker_v2[(bucket_dwalker_v2['stock_qty']==0)&(bucket_dwalker_v2['color']=='OW')&(bucket_dwalker_v2['scskey']==i)]

,shopkey,weekkey,partkey,scskey,brand,season,partcode,color,siz,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,week_sale_amt,stock_qty,sale_amt,gender
208,X10056,2020-03-16,X20SDXSHA5011,X20SDXSHA5011OW270,X,20S,DXSHA5011,OW,270,0,3,1,3,139000.00000,0,417000.00000,X
244,X10056,2020-07-06,X20SDXSHA5011,X20SDXSHA5011OW270,X,20S,DXSHA5011,OW,270,0,4,1,4,139000.00000,0,556000.00000,X
506,X10056,2019-12-09,X20SDXSHA5011,X20SDXSHA5011OW270,X,20S,DXSHA5011,OW,270,0,1,1,1,139000.00000,0,139000.00000,X
2223,X50077,2020-11-02,X20SDXSHA5011,X20SDXSHA5011OW270,X,20S,DXSHA5011,OW,270,0,8,3,8,375930.00000,0,993130.00000,X
2257,X50077,2020-04-27,X20SDXSHA5011,X20SDXSHA5011OW270,X,20S,DXSHA5011,OW,270,0,4,1,4,129000.00000,0,498200.00000,X
2401,X50077,2020-02-24,X20SDXSHA5011,X20SDXSHA5011OW270,X,20S,DXSHA5011,OW,270,0,3,2,3,240200.00000,0,369200.00000,X
2669,X50006,2020-04-20,X20SDXSHA5011,X20SDXSHA5011OW270,X,20S,DXSHA5011,OW,270,0,8,1,8,131980.00000,0,1091080.00000,X
3420,X50128,2019-12-09,X20SDXSHA5011,X20SDXSHA5011OW270,X,20S,DXSHA5011,OW,270,0,1,1,1,139000.00000,0,139000.00000,X
5263,X50101,2020-03-09,X20SDXSHA5011,X20SDXSHA5011OW270,X,20S,DXSHA5011,OW,270,2,21,4,21,514330.00000,0,2784160.00000,X
6971,X50131,2019-12-16,X20SDXSHA5011,X20SDXSHA5011OW270,X,20S,DXSHA5011,OW,270,0,2,1,2,139000.00000,0,278000.00000,X
